** This project is about analysing the hacker news post and identifying the most popular or most attracted post. **

In [1]:
from csv import reader as r
open_file = open("hacker_news.csv")
read_file = r(open_file)
hn=list(read_file)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In [2]:
headers=hn[:1]
hn=hn[1:]
print(headers)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']]
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


In [3]:
ask_posts=[]
show_posts=[]
other_posts=[]
for row in hn:
    title=row[1]
    if (title.lower()).startswith('ask hn'):
        ask_posts.append(row)
    elif (title.lower()).startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


In [4]:
total_ask_comments=0
tot_comments=0
for row in ask_posts:
    num_comments=int(row[4])
    total_ask_comments+=num_comments
    tot_comments+=1
avg_ask_comments=total_ask_comments/tot_comments
print(avg_ask_comments)


14.038417431192661


In [5]:
total_show_comments=0
tot_comments=0
for row in show_posts:
    num_comments=int(row[4])
    total_show_comments+=num_comments
    tot_comments+=1
avg_show_comments=total_show_comments/tot_comments
print(avg_show_comments)

10.31669535283993


*** Based on the above analysis, the ask posts receive more comments than show posts. ***

In [8]:
import datetime as dt
result_list=[]
for post in ask_posts:
    created_at=post[6]
    num_comments = post[4]
    result_list.append([created_at,num_comments])
# print(result_list)
counts_by_hour={}
comments_by_hour={}

for row in result_list:
    created_at=row[0]
    num_comments=int(row[1])
    date_dt=dt.datetime.strptime(created_at,"%m/%d/%Y %H:%M")
    date_hr=date_dt.hour
    if date_hr not in counts_by_hour:
        counts_by_hour[date_hr]=1
        comments_by_hour[date_hr]=num_comments
    elif date_hr in counts_by_hour:
        counts_by_hour[date_hr]+=1
        comments_by_hour[date_hr]+=num_comments
        
print(counts_by_hour)
print(comments_by_hour)

avg_by_hour=[]
for hour in counts_by_hour:
    no_of_posts=int(counts_by_hour[hour])
    no_of_comments=int(comments_by_hour[hour])
    avg_comments=round((no_of_comments/no_of_posts),2)
    avg_by_hour.append(list([hour,avg_comments]))

print(avg_by_hour)
    
swap_avg_by_hour=[]

for row in avg_by_hour:
    swap_avg_by_hour.append(list([row[1],row[0]]))
    
print(swap_avg_by_hour)

sorted_swap=sorted(swap_avg_by_hour,reverse=True)

for row in sorted_swap:
    hr_dt=dt.datetime.strptime(str(row[1]),"%H")
    hr=hr_dt.strftime("%H")
    print("{} {:.2f} average comments per cost",hr,row[1])
    

{0: 55, 1: 60, 2: 58, 3: 54, 4: 47, 5: 46, 6: 44, 7: 34, 8: 48, 9: 45, 10: 59, 11: 58, 12: 73, 13: 85, 14: 107, 15: 116, 16: 108, 17: 100, 18: 109, 19: 110, 20: 80, 21: 109, 22: 71, 23: 68}
{0: 447, 1: 683, 2: 1381, 3: 421, 4: 337, 5: 464, 6: 397, 7: 267, 8: 492, 9: 251, 10: 793, 11: 641, 12: 687, 13: 1253, 14: 1416, 15: 4477, 16: 1814, 17: 1146, 18: 1439, 19: 1188, 20: 1722, 21: 1745, 22: 479, 23: 543}
[[0, 8.13], [1, 11.38], [2, 23.81], [3, 7.8], [4, 7.17], [5, 10.09], [6, 9.02], [7, 7.85], [8, 10.25], [9, 5.58], [10, 13.44], [11, 11.05], [12, 9.41], [13, 14.74], [14, 13.23], [15, 38.59], [16, 16.8], [17, 11.46], [18, 13.2], [19, 10.8], [20, 21.52], [21, 16.01], [22, 6.75], [23, 7.99]]
[[8.13, 0], [11.38, 1], [23.81, 2], [7.8, 3], [7.17, 4], [10.09, 5], [9.02, 6], [7.85, 7], [10.25, 8], [5.58, 9], [13.44, 10], [11.05, 11], [9.41, 12], [14.74, 13], [13.23, 14], [38.59, 15], [16.8, 16], [11.46, 17], [13.2, 18], [10.8, 19], [21.52, 20], [16.01, 21], [6.75, 22], [7.99, 23]]
{} {:.2f} ave